# Adaptive bw KDE Exploration

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle as pkl
import matplotlib.pyplot as plt
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

sys.path.append('/Users/cmg/dev/spatial-assocr/kde')
# from location_project import kde_2d  # adaptive bw KDE 
%load_ext autoreload
%autoreload 2 # auto reload changes to kde code for faster prototyping

%matplotlib inline
sys.setrecursionlimit(10000)

## Load & format filtered data

In [2]:
# read in the filtered data
filepath = os.path.join("..", "data", "mpp_month0a_month1b_n50.csv")
mpp = pd.read_csv(filepath)
mpp.head()

,id,m,lon,lat
0,11.0,a,-117.92895,33.61891
1,11.0,a,-117.67199,33.60002
2,11.0,a,-117.92895,33.61891
3,11.0,a,-117.67199,33.60002
4,11.0,a,-117.67199,33.60002


In [3]:
# reorder the columns... kde_2d expects [id, lon, lat]
data = mpp[['id', 'lon', 'lat']]
data.head()

,id,lon,lat
0,11.0,-117.92895,33.61891
1,11.0,-117.67199,33.60002
2,11.0,-117.92895,33.61891
3,11.0,-117.67199,33.60002
4,11.0,-117.67199,33.60002


In [4]:
# convert the pandas data frame to array of arrays
# i.e., <np.array [[user_id, lon, lat], ... ] >
df = data.values
df

array([[ 1.1000000e+01, -1.1792895e+02,  3.3618910e+01],
       [ 1.1000000e+01, -1.1767199e+02,  3.3600020e+01],
       [ 1.1000000e+01, -1.1792895e+02,  3.3618910e+01],
       ...,
       [ 1.7282000e+04, -1.1795213e+02,  3.3684750e+01],
       [ 1.7282000e+04, -1.1795213e+02,  3.3684750e+01],
       [ 1.7282000e+04, -1.1795197e+02,  3.3684750e+01]])

## New code for adaptive bw kde

In [5]:
from scipy.special import logsumexp
from scipy.spatial import KDTree
from scipy.stats import norm

KM_TO_LON = 0.010615
KM_TO_LAT = 0.008989

def learn_nearest_neighbors_bandwidth(
        sample_points, k=5, lon_to_km=KM_TO_LON, lat_to_km=KM_TO_LAT, min_bw=0.01
    ):
    """
    Learning the bandwidth
    :param data:
    :param k:
    :return:
    """
    k = np.min([k, sample_points.shape[0] - 1])

    bandwidths = []

    anchor_point = np.amin(sample_points, 0)
    dists = sample_points - anchor_point
    
    dists[:, 0] /= lon_to_km
    dists[:, 1] /= lat_to_km

    dists +=  np.random.random(dists.shape) * 0.0000001
    
    # Building the k-d tree
    tree = KDTree(dists, leafsize=500)

    for i in range(dists.shape[0]):
        (neighbors_dists, neighbors_indexes) = tree.query(dists[i, :], k + 1)

        if neighbors_dists[-1] <= min_bw:
            bandwidths.append(min_bw)  # bandwidth can't be less than 1 meter
        else:
            bandwidths.append(neighbors_dists[-1])

    print('Done training bandwidths')
    return np.array(bandwidths)


def log_pdf(query_point, kde_data):
    """
     INPUT:
    -------
        :param query_point:
        :param sample_point:

     OUTPUT:
    --------
        1. log_pdf: <float> The log pdf value
    """

    x_dist = query_point[0] - kde_data[:, 0]
    y_dist = query_point[1] - kde_data[:, 1]
    log_pdf_x = norm.logpdf(x_dist, loc=0, scale=kde_data[:, 2]*KM_TO_LON)
    log_pdf_y = norm.logpdf(y_dist, loc=0, scale=kde_data[:, 2]*KM_TO_LAT)
    
    return logsumexp(log_pdf_x + log_pdf_y) #- np.log( kde_data.shape[0] )


##### USAGE - from Moshe
# points = mpp.loc[:, ['lon', 'lat']].values
# bw_pop = learn_nearest_neighbors_bandwidth(points)
# kde_data = np.hstack([points, np.atleast_2d(bw_pop).T])
# delta = 0.01
# x = np.arange(mpp.lon.min(), mpp.lon.max(), delta)
# y = np.arange(mpp.lat.min(), mpp.lat.max(), delta)

# m = np.zeros([y.shape[0], x.shape[0]])
# for i, lon in enumerate(x):
#   for j, lat in enumerate(y):
#       m[j, i] = log_pdf([lon, lat], kde_data)

## Fit an adaptive bandwidth KDE to the data
Do this for the entire sample (i.e., a population model). 

In [6]:
# learn the bandwidth for each point & save to file (it's expensive)
# filepath = os.path.join("..", "data", "bw_pop_k10_month0a_month1b_n50.npy")
filepath = os.path.join("..", "data", "bw_pop_k5_month0a_month1b_n50.npy")
bw_pop = learn_nearest_neighbors_bandwidth(df[:, 1:3], k=5, min_bw=0.01)
np.save(filepath, bw_pop)
bw_pop = np.load(filepath)


kde_data = np.hstack([df[:, 1:3], np.atleast_2d(bw_pop).T])
kde_data

Done training bandwidths


array([[-1.1792895e+02,  3.3618910e+01,  1.0000000e-02],
       [-1.1767199e+02,  3.3600020e+01,  1.0000000e-02],
       [-1.1792895e+02,  3.3618910e+01,  1.0000000e-02],
       ...,
       [-1.1795213e+02,  3.3684750e+01,  1.0000000e-02],
       [-1.1795213e+02,  3.3684750e+01,  1.0000000e-02],
       [-1.1795197e+02,  3.3684750e+01,  1.0000000e-02]])

In [7]:
# format the data for the KDE class; equally weight points
# <np.array [[user_id, lon, lat, bw, weight], ... ] >
# pop = np.append(df, np.reshape(bw_pop, (len(df), 1)), 1)
# pop = np.append(pop, np.ones((len(df), 1)), 1)

# # create the KDE
# kde_pop = kde_2d.KDE(pop)

In [8]:
# compute the log pdf for a given point
# < np.array [lon, lat] >
# kde_pop.log_pdf(df[0][1:])

In [9]:
# sample from the KDE
# must pass in the data to do so b/c its a nonparametric model
# kde_pop.sample_from_kde(pop)

### Contour plot of the density

In [10]:
# compute log pdf values over grid
delta = 0.01
x = np.arange(-118.2, -117.5, delta)  # longitude
y = np.arange(33.4, 34, delta)  # latitude
X, Y = np.meshgrid(x, y)
pts = np.vstack([X.ravel(), Y.ravel()]).T
z = np.apply_along_axis(log_pdf, 1, pts, kde_data=kde_data)

# filepath = os.path.join("..", "data", "kde_pop_k10_month0a_month1b_n50.npy")
# filepath = os.path.join("..", "data", "kde_pop_k5_month0a_month1b_n50.npy")
# np.save(filepath, z)
# z = np.load(filepath)

In [11]:
# save pandas to pickle to debug more later
filepath = os.path.join("..", "data", "kde_pop_k5_month0a_month1b_n50.pkl")
out = pd.DataFrame({'lon': pts[:,0], 'lat': pts[:,1], 'lpdf': z})
# out.to_pickle(filepath)
# out = pd.read_pickle(filepath)
out.head()

,lon,lat,lpdf
0,-118.20,33.4,-55.348387
1,-118.19,33.4,-54.123890
2,-118.18,33.4,-52.924640
3,-118.17,33.4,-51.750636
4,-118.16,33.4,-50.601872


In [12]:
# create the contour plot
heat = [
    go.Heatmap(
        z = out.lpdf,
        x = out.lon,
        y = out.lat,
        colorscale=[
            [1.0, 'rgb(165,0,38)'], 
            [0.8888888888888888, 'rgb(215,48,39)'], 
            [0.7777777777777778, 'rgb(244,109,67)'], 
            [0.6666666666666666, 'rgb(253,174,97)'], 
            [0.5555555555555556, 'rgb(254,224,144)'], 
            [0.4444444444444444, 'rgb(224,243,248)'], 
            [0.3333333333333333, 'rgb(171,217,233)'], 
            [0.2222222222222222, 'rgb(116,173,209)'], 
            [0.1111111111111111, 'rgb(69,117,180)'], 
            [0.0, 'rgb(49,54,149)']
        ]
    )
]

layout = go.Layout()

fig = go.Figure(data=heat, layout=layout)
iplot(fig, show_link=False)

## Sample from the above KDE

In [13]:
def sample_from_kde(data):
    """
    Sampling a point from data. This is done separately from the actual
    object because in the sampling I will usually change the weights all the
    time (because that's how the sampling is done) and I don't want to split
    the ball tree every time (it's expensive). Sampling from KDE is done in
    the following way:

        1. We sample a point from data according to the weights.
        2. We sample a point from MVN with the mean as the sampled point from
            (1) and the bw of the point from (1)


     INPUT:
    -------
        1. data: The observed/train events.
                The data is in the format of
                np.array([[user_id, lon, lat, bandwidth, weight], ... ])
                where each line is a different point.
                NOTE: We assume the weights sum to 1!!!!

     OUTPUT:
    --------
        1. sampled_point: [lon, lat]
    """
    sampled_point_index = np.random.choice(data.shape[0], p=data[:, -1])
    sampled_point = data[sampled_point_index, :]

    sample_lon = np.random.normal(sampled_point[1], sampled_point[3] * KM_TO_LON)
    sample_lat = np.random.normal(sampled_point[2], sampled_point[3] * KM_TO_LAT)

    return [sample_lon, sample_lat]

In [14]:
pop = np.append(df, np.reshape(bw_pop, (len(df), 1)), 1)
pop = np.append(pop, np.ones((len(df), 1)), 1)
pop[:, -1] /= np.sum(pop[:, -1])
pop

array([[ 1.10000000e+01, -1.17928950e+02,  3.36189100e+01,
         1.00000000e-02,  3.80314901e-05],
       [ 1.10000000e+01, -1.17671990e+02,  3.36000200e+01,
         1.00000000e-02,  3.80314901e-05],
       [ 1.10000000e+01, -1.17928950e+02,  3.36189100e+01,
         1.00000000e-02,  3.80314901e-05],
       ...,
       [ 1.72820000e+04, -1.17952130e+02,  3.36847500e+01,
         1.00000000e-02,  3.80314901e-05],
       [ 1.72820000e+04, -1.17952130e+02,  3.36847500e+01,
         1.00000000e-02,  3.80314901e-05],
       [ 1.72820000e+04, -1.17951970e+02,  3.36847500e+01,
         1.00000000e-02,  3.80314901e-05]])

In [15]:
sample_from_kde(pop)

[-117.92739480414724, 33.61102517005087]